# Rouspaumcation

Simulation program designed to solve problem of dynamic routing, space and spectrum allocation (RSSA) of unicast demands in flex-grid network with assistive sotrage.

Needed data set:
- .net file with network dimensions: number of nodes, links and adjacent matrix,
- .pat file with k candidate paths (k can be set up in code),
- .spec file with number of slices needed for each path in .pat file,
- .dem files with demands (income iteration, source, target, bitrate and duration).

# Project

The aim of the project is to propose new created method to solve RSSA problem (Dynamic Routing Space and Spectrum Allocation) with assistive storage. The method consists in creating candidate demand sets for each spectrally-spatially resources and selecting those demands that limit available resources for the fewest other demands. It has been described in detail through the criteria for creating such sets, and a simulation program has been implemented to test it for different traffic volumes. This gives the opportunity to measure how beneficial it is to use assistive storage in various cases.

## Introduction

Many observations show that Internet traffic is growing faster and faster. The introducing technologies will bring about many changes. A good example is 5G technology, which in 2023 will reach speeds around 13 times faster than mobile traffic in 2018 [1].

The technology that can cope with such an increase in network bandwidth demand is concept of spectrally-spatially flexible optical networks (SS-FONs). It implements space division multiplexing (SDM) [2] with elastic optical networks (EON), what brings benefits like enormous increase in transmission capacity [3, 4]. It is also opportunity to manage network resource in more flexible way.

It works by discretization the optical spectrum in frequency slots (FSs) of 12.5 GHz width [5]. By concatenating multiple adjacent Sub-Channels (Sb-Chs) it is possible to form Super-Channel (SCh) that enables ultra-high bit-rate transmissions [6, 7].

According to [1], the global average broadband speed is growing, from 45.9 Mbps in 2018 to 110.4 Mbps in 2023. This is one of the key indicators of increased IP traffic. This raises the need to find better methods of planning unicast traffic using existing infrastructure. The attempt to meet these requirements is working on dynamic routing, spectrum and space allocation (RSSA) problem. The aim is to find for each request a routing path, spatial cores and optical channel. It is also possible to determine a modulation level and handle with routing, modulation level, space and spectrum allocation (RMLSSA) [8], but in this paper modulation level is pre-determined. Many publication consider also routing and spectrum allocation (RSA) problem. 

RSSA problem is to find a lightpath in SS-FONs scenario, what requires to focus on spectrum and space allocation in the same time for incoming requests. There are many ways to solve this problem, what we present in the next chapters. We want to create new method basing on probability of using a specific resource of space and spectrum for unicast requests. In this paper we want to describe proposed algorithm and present its implementation. Next we want to conduct research and compare results with results obtained by the SSA algorithm.

## Related works

The subject of this article is quite a new field of Internet traffic engineering. Nevertheless, many works have been prepared proposing various approaches to the problem of allocation of the discussed resourches.

In many works RSSA problem is studied with using integer linear programming (ILP) [4, 7, 9, 10] Formulating this problem as ILP model provides to solve optimization problem with decision variables. Hovewer, this technique is low scalability. Mathematical programming (MP) methods for larger instances of complex problems don't provide results in a reasonable time [3].

Another way is using heuristic or metaheuristic algorithms, which are faster, but not always reach optimal solution of a problem.

Fixed-Alternate routing and First-Fit frequency assignment (FA-FF) algorithm is proposed in [11]. In [7] authors propose effective heuristic method called Adaptive Frequency Assignment - Collision Avoidance (AFA-CA). In this method the main aim is to select the sequence of processed demands in order to minimize the spectrum use. In [12] authors propose two algorithms solving RSSA-DPP (Dedicated Path Protection) problem using evolutionary algorithm and tabu search. Three different heuristic algorithms called spectrum waste base (SWB) algorithm, intentional spectrum waste with sequential approach (ISW-S) algorithm and intentional spectrum waste with accumulative approach (ISW-A) are proposed in [8] using spectrum waste metric. Another heuristic is developed in [13], basing on the greedy randomized adaptive search procedure authors propose GRASP metaheuristic approach. They also propose adaptive search procedure (SA) to solve larger instances of the RSSA problem. In [14] authors propose another algorithm using greedy heuristic. Another algorithms are: a variable-group-base (MVG) algorithm [15], a minimum-block-generated flexible-grouping-based (MBFG) spectrum assignment algorithm [16] and an SLA-provisioning algorithm (SADQ) that employs multilevel differentiation to provision the proposed service level agreement (SLA) parameters [17].

The last way to solve discussed problem is to use the techniques of artificial intelligence (AI). In [18] described is using AI in optical networks, what can improve the configuration and operation of network devices, optical performance monitoring and quality of transmission (QoT) estimation. AI systems gives opportunities for automating operations and introducing intelligence decisions by planning use and management of network resources.

## Problem definition

In this section we want to present a way to describe RSSA problem as a mathematical model. There are two cases: static network parameters and variables of resources that are used by incoming demands.

Static network parameters refers to network dimensions. The SS-FON can be modeled as a directed graph $G = (V, E)$, where $V$ is a set of network nodes, $E$ is a set of directed physical network links. On each link there is a set of available spatial modes $K$ and on each mode there is a set of frequency slices $S$. Channel in such a flex-grid SDM network is a set of adjacent slices on the same fiber mode.

For each pair of two network nodes $(v_i, v_j)$ there is a set of candidate routing paths $P_{ij}$. Constant $\delta_{epij}$ is equal to $1$ if a link $e$ is a part of a candidate path $p$ between nodes $v_i$ and $v_j$. Otherwise $\delta_{epij}$ is equal to $0$.

To describe the occurrence of incoming requests, we will treat time as successive iterations $n = 1, 2, ..., N$. Each demand from set $D$ is described by its iteration of arrival $a_d$, source node $s_d$, destination node $t_d$, bit-rate $h_d$ (in Gbps) and duration $l_d$ (number of iterations). On the basis of made determinations, there is a set of candidate paths for the each request $d$ described as $P_d = P_{s_d t_d}$ and constant $\delta_{epd} = \delta_{ep s_d t_d}$.

For each link $e$ there is a function $f_e(h)$, which returns number of needed frequency slices for a given bit-rate $h$ in Gbps. For each network node $v$ there is a buffer that is limited by the maximum number of requests $B_v$ it can remember.

The solution of the problem is to determine how long each request should be stored in a buffer, through which routing path should be sent and which spectrally-spatially resources should be used.

If demand $d$ after its arrival iteration $a_d$ is not stored in its source node buffer but starts to be processed immediately, then $b_d = 0$. Otherwise $b_d$ is equal to number of iterations, that demand $d$ is stored in this buffer. That means, that each demand $d$ begins to be processed in iteration $a_d + b_d$ and ends to be processed at iteration $a_d + b_d + l_d$. If demand $d$ is rejected from a buffer without processing, then $r_d = 1$, otherwise $r_d = 0$. At each iteration number of stored demands in buffer on node $v$ can not be greater than $B_v$. This constraint is defined in (\ref{aqn:buffer}).

When demand $d$ starts to be processed, the routing path is selected and free resources are allocated. In this paper we consider non-bifurcated flows. If for demand $d$ chosen is path $p \in P_d$, then $x_{dp} = 1$, otherwise $x_{dp} = 0$. This constraint is defined in (\ref{aqn:onepath}). On mentioned path $p$ allocated resources form a channel with adjacent slices on selected fiber mode. This channel is determined by variable $c_{dks}$, which is equal to 1 if demand $d$ uses fiber mode $k$ and adjacent frequency slices beginning with slice $s$, otherwise $c_{dks} = 0$. Constraint about allocation slices on only one mode is defined in (\ref{aqn:onemode}).

On given network link $e$ if there is demand $d$ that uses channel formed by spatial mode $k$ and adjacent slices beginning with slice $s$, then all slices from $s$ to $s + f_e(h_d) + 1$ are used, because $f_e(h_d)$ is equal to number of slices needed for demand $d$ basing on its bit-rate. There is one additional slice because of guard-band. That means, if on that link $e$ and mode $k$ demand $d$ uses slice $s$, there is slice $s' \in \left< s-\left( f_e(h_d) + 1 \right); s \right>$ which is beginning of formed channel for this demand, so $\sum_{s' \in \left< s-\left( f_e(h_d) + 1 \right); s \right>} c_{dks'} = 1$.

### Nomenclature
#### Sets and indices
$v \in V$ : Network nodes

$e \in E$ : Network physical links

$k \in K$ : Spatial modes of a single link

$s \in S$ : Frequency slices of a single mode

$p \in P_{ij}$ : Candidate routing paths between $v_i$ and $v_j$

$n \in N$ : Iterations

$d \in D$ : Demands

$p \in P_d$ : Candidate paths for demand $d$

#### Constants and functions
$s_d$ : Source node of demand $d$

$t_d$ : Target node of demand $d$

$h_d$ : Bit-rate in Gbps of demand $d$

$a_d$ : Iteration of demand $d$ arrival

$l_d$ : Duration of demand $d$ (number of iterations)

$\delta_{epij}$ : $=1$ if $\exists_{ p \in P_{ij}} e \in p$; $0$, otherwise

$\delta_{epd}$ : $=1$ if $\exists_{ p \in P_d} e \in p$; $0$, otherwise

$B_v$ : Number of demands that can be stored in assistive storage on node $v$

$f_e(h)$ : Function that returns number of needed slices for bit-rate $h$ on a link $e$

#### Variables
$r_d$ : $=1$ if demand $d$ is rejected; $0$, otherwise

$b_d$ : Number of iteration that demand $d$ is stored in its source buffer

$x_{dp}$ : $=1$ if for demand $d$ chosen is path $p \in P_d$'; $0$, otherwise

$c_{dks}$ : $=1$ if demand $d$ uses fiber mode $k$ and frequency slices beginning with slice $s$; $0$, otherwise

$y_{neks}$ : $=1$ if at iteration $n$ on physical link $e$ frequency slice $s$ on  mode $k$ is in use; $0$, otherwise; auxiliary variable


Variable $c_{dks}$ is related to auxiliary variable $y_{neks}$, which is equal to 1 if at iteration $n$ on physical link $e$ frequency slice $s$ on spatial mode $k$ is in use, otherwise $y_{neks} = 0$. At each iteration single frequency slice can be allocated to only one channel for only one demand, what is defined in (\ref{aqn:onechannel}).

The aim of the considered optimization problem is to allocate as much demands as it is possible. Objective functions we want to minimize are demand blocking probability (DBP) defined in (\ref{aqn:dbp}) and bit-rate blocking probability (BBP) defined in (\ref{aqn:bbp}).

$$
\begin{equation}
\begin{split}
\forall_{n \in N} \forall_{v \in V} \vert \{ d \in D: s_d = v \land b_d > 0 \\
\land a_d + b_d = n \land r_d = 0 \} \vert \leq B_v
\end{split}
\label{aqn:buffer}
\end{equation}
$$

$$
\begin{equation}
\forall_{d \in D} \sum_{p \in P_d} x_{pd} = 1
\label{aqn:onepath}
\end{equation}
$$

$$
\begin{equation}
\forall_{d \in D} \exists!_{k \in K} \sum_{s \in S} c_{dks} = 1
\label{aqn:onemode}
\end{equation}
$$

$$
\begin{equation}
\begin{split}
\forall_{n \in N} \forall_{e \in E} \forall_{k \in K} \forall_{s \in S} y_{neks} = \vert \{ d \in D : \\
r_d = 0 \\
\land a_d + b_d \leq n \leq a_d + b_d + l_d \\
\land \left( \exists!_{p \in P_d} x_{pd} = 1 \land \delta_{epd} = 1 \right) \\
\land \sum_{s' \in \left< s-\left( f_e(h_d) + 1 \right); s \right>} c_{dks'} = 1 \} \vert \\
\in \{0, 1\}
\end{split}
\label{aqn:onechannel}
\end{equation}
$$

$$
\begin{equation}
BBP = \frac{\sum_{d \in D} r_d \cdot h_d}{\sum_{d \in D} h_d}
\label{aqn:bbp}
\end{equation}
$$

$$
\begin{equation}
DBP = \frac{\sum_{d \in D} r_d}{|D|}
\label{aqn:dbp}
\end{equation}
$$

## Description of the method

In this section we propose method that consists of creating candidate request sets for resources. We present new variables in each step with mathematical formulas basing on assumed network model.


### Method steps

General idea of proposed method is to find at subsequent iterations for each spatial mode $k$ and each frequency slice $s$ on each physical link $e$ how many demands can use this resource as beginning slice of its channel. That means, that for each space and spectrum resource we need to find a list of candidate demands $D_{eks} \subset D$. If tested resource determined by $e, k, s$ is currently in use, then current list of candidate demands $D_{eks}$ is empty. Moreover, after preparing lists $D_{eks}$ for each resource, for each demand $d$ that could start processing in current iteration there are prepared candidate pairs of paths and channels described as $c'_{d}$.

We introduce also two auxiliary variables: $q_d$ that is equal to $1$ if demand $d$ is processing, otherwise is equal to $0$; and $m_d$ that is equal to $1$ if demand $d$ processing is completed, otherwise is equal to $0$.

At a given iteration $n$ included in preparing sets $D_{eks}$ demands that are not rejected ($r_d = 0$), not processing ($q_d  = 0$), not completed ($m_d = 0$) and can be processed in current $n$-th iteration($n = a_d + b_d$). Considered demands can use physical link $e$, what can be described as $\sum_{p \in P_d} \delta_{epd} > 0$. It is also necessary to check if there are enough unused adjacent slices, what is determined in (\ref{aqn:enoughtslices}). It is also needed to ensure slices of guard-bands - slice $s-1$ and slice $s+1$, what is determined in (\ref{aqn:guardband}).

$$
\begin{equation}
\label{aqn:enoughtslices}
s+f_e(h_d) \leq |S| \land \sum_{s' \in \left< s; s+f_e(h_d) \right>} y_{neks'} = 0
\end{equation}
$$

$$
\begin{equation}
\label{aqn:guardband}
\begin{split}
\left(s > 1 \implies y_{nek(s-1)} = 0 \right) \\
\land \left( s+f_e(h_d) < |S| \implies y_{nek(s+1)} = 0 \right)
\end{split}
\end{equation}
$$

Basing on the presented assumptions, it is possible to prepare at given $n$-th iteration sets of candidate demands $D_{eks}$ for each space and spectrum resource on each link by formula (\ref{aqn:deks}).

Knowing set $D_{eks}$ it is possible to find candidate pairs of paths and channels, what is determined in (\ref{aqn:cd}). This set respects that candidate resources for a demand $d$ have to be provided on all physical links included in the candidate path. Set $D_{eks}$ treats links forming paths separately.

Having information about candidate paths and channels for each demand, it is possible to prepare new set of candidate demands for all frequency slices included in each spatial mode on each physical link respecting, that for each demand all resources have to be allocated all along a path in the same way. This new set is $D'_{eks}$ and is determined in (\ref{aqn:improveddeks}).

$$
\begin{equation}
\label{aqn:deks}
\begin{split}
\forall_{e \in E} 
\forall_{k \in K} 
\forall_{s \in S} 
D_{eks} = 
\end{split}
\begin{cases}
\emptyset, \text{if } y_{neks} = 1\\
\{ d \in D:
r_d = 0 
\land q_d = 0 
\land m_d = 0 
\land n = a_d + b_d \\
\hspace{1.5cm} \land \sum_{p \in P_d} \delta_{epd} > 0 
\land s+f_e(h_d) \leq |S| 
\land \sum_{s' \in \left< s; s+f_e(h_d) \right>} y_{neks'} = 0 \\
\hspace{1.5cm} \land \left(s > 1 \implies y_{nek(s-1)} = 0 \right) 
\land \left( s+f_e(h_d) < |S| \implies y_{nek(s+1)} = 0 \right)
\}, \text{otherwise}
\end{cases}
\end{equation}
$$

$$
\begin{equation}
\label{aqn:cd}
\begin{split}
c'_d = \{ (p, k, s) \in P_d \times K \times S : \forall_{e \in p} d \in D_{eks} \}
\end{split}
\end{equation}
$$

$$
\begin{equation}
\label{aqn:improveddeks}
\begin{split}
D'_{eks} = \{ d \in D_{eks} : \exists_{p \in P_d} (p, k, s) \in c'_d \}
\end{split}
\end{equation}
$$

In this method we find resource with the least numerous set of candidate demands (but not empty). If we assign one demand to this resource, number of other demands that now can not use this resource is the smallest. If there are more than one resources with the least numerous set of candidate demands, they are ordered ascending by delay of demand with the smallest delay ($l_d$) included in each set, because demands with shortest delay affect the least of resources in subsequent iterations, and after ordering chosen is a first set. Let mark this set as $D'_{e'k's'}$ and determine by (\ref{aqn:smallestdeks}).

$$
\begin{equation}
\label{aqn:smallestdeks}
\begin{split}
\forall_{e \in E} 
\forall_{k \in K} 
\forall_{s \in S} 0 < | D'_{e'k's'} | \leq | D'_{eks} | \\
\land \left( | D'_{e'k's'} | = | D'_{eks} | \implies \forall_{d \in D'_{eks}}\exists_{d' \in D'_{e'k's'}} l_d' \leq l_d \right) \\
\land | D'_{e'k's'} | > 0 \\
\land | D'_{eks} | > 0
\end{split}
\end{equation}
$$

After choosing this resource $D'_{e'k's'}$, demand with smallest delay is indicated from set of candidate demands for this resource. Let the index of this demand be $d'$ and is determined in (\ref{aqn:shortestd}).

$$
\begin{equation}
\label{aqn:shortestd}
\begin{split}
\forall_{d \in D'_{e'k's'}} l_{d'} \leq l_d
\end{split}
\end{equation}
$$

In this way first demand to allocate resources at given iteration $n$ is chosen. There can be many candidate paths $P'$ for this demand which includes physical link $e'$ and resources determined by $k'$ and $s'$ (\ref{aqn:conflictp}). In this case should be chosen this path $p' \in P'$, which has the least effect on other candidate demands on resources on this path, what is precised in (\ref{aqn:bestpath}) in two ways (recommended is the second one, because each demand in this way is counted only once).

$$
\begin{equation}
\label{aqn:conflictp}
\begin{split}
P' = \{p \in P_{d'} : (p, k', s') \in c'_{d'} \land \delta_{e'pd'} = 1\}
\end{split}
\end{equation}
$$

$$
\begin{equation}
\label{aqn:bestpath}
\begin{split}
\forall_{p \in P'} \sum_{e \in E} \left( \delta_{ep'd'} \cdot | D'_{ek's'}| \right) \leq \sum_{e \in E} \left( \delta_{epd'} \cdot | D'_{ek's'}| \right)
\\
or
\\
\forall_{p \in P'} \left| \bigcup_{e \in E : \delta_{ep'd'} = 1}  D'_{ek's'} \right| \leq \left| \bigcup_{e \in E : \delta_{epd'} = 1}  D'_{ek's'} \right|
\end{split}
\end{equation}
$$

Knowing path $p'$, spatial mode $k'$ and frequency slice $s'$ is enough to allocate resources to demand $d'$ (\ref{aqn:allocating}).

$$
\begin{equation}
\label{aqn:allocating}
\begin{split}
x_{d'p'} = 1 \\
c_{d'k's'} = 1 \\
\forall_{n' \in \left < a_{d'} + b_{d'}; a_{d'} + b_{d'}+ l_{d'} \right>} y_{n'e'k's'} = 1 \\
q_{d'} = 1
\end{split}
\end{equation}
$$

Next, all resources, that demand $d'$ uses, should be found and their sets of candidate demands $D_{eks}$ should be emptied. That operation is described in (\ref{aqn:empting}). This demand should be also removed from all other sets $D_{eks}$.

$$
\begin{equation}
\label{aqn:empting}
\begin{split}
\forall_{d \in D : \delta_{ep'd'} = 1} D_{ek's'} := \emptyset \\
\forall_{d \in D : \delta_{ep'd'} = 0} D_{ek's'} := D_{ek's'} \setminus d'
\end{split}
\end{equation}
$$

After that operation, next demand should be found in the same way. Repeating steps (\ref{aqn:cd} ) - (\ref{aqn:empting}) as long as there are possibly found demands $d'$.

The next step of this method is to use of assistive storage for unprocessed unrejected demands. Let mark a set of these demands as $d'' \in D'' \subset D$. Each demand $d''$ has no allocated recources, so $q_{d''} = 0$ and at given $n$-th iteration is stored in a buffer or arrived at this iteration, so $a_{d''} + b_{d''} = n$. We can describe set $D''$ by (\ref{aqn:bufferr}).

$$
\begin{equation}
\label{aqn:bufferr}
\begin{split}
D'' = \{ d \in D : r_d + q_d + m_d = 0 \land a_d + b_d + l_d = n \} \\
\forall_{d \in D'''} m_d = 1
\end{split}
\end{equation}
$$

Next, for each network node $v$ it is checked if there are enough storage for demands arrived on this node. Set of stored or arrived at this iteration demands on node $v$ is marked as $D''_v \subset D''$ and is determined in (\ref{aqn:nodebuffer}).

$$
\begin{equation}
\label{aqn:nodebuffer}
\begin{split}
D''_v = \{ d \in D'' : a_d = v \}
\end{split}
\end{equation}
$$

Buffer on network node $v$ can store at most $B_v$ demands. If $|D''_v| > B_v$, it is necessary to decide, which demands from $D''_v$ should be rejected. In this method, way to do it is to reject $B_v - |D''_v|$ demands $d'' \in D''_v$ with the greatest value of delay $l_d$. This procedure can be described as (\ref{aqn:rejecting}).

$$
\begin{equation}
\label{aqn:rejecting}
\begin{split}
\forall_{v \in V} \exists_{l_v \in \mathbb{N}} \left| \{ d \in D''v : l_d \geq l_v \} \right| = B_v - |D''_v| \\
\forall_{v \in V} \forall_{d \in D''_v : l_d \geq l_v} r_d = 1
\end{split}
\end{equation}
$$

For not rejected demand value of number of iterations that this demand is stored in a buffer is inceasing by 1: $b_d := b_d + 1$.

Last step of this method is to release resources, which were used by demands that end processing in a given iteration $n$. Set of these demands is marked by $D'''$ and determined in (\ref{aqn:ended}). Each demand from this set should be marked as completed by changing value of $m_d$.

$$
\begin{equation}
\label{aqn:ended}
\begin{split}
D''' = \{ d \in D : r_d = 0 \land a_d + b_d + l_d = n \} \\
\forall_{d \in D'''} m_d = 1
\end{split}
\end{equation}
$$

Demand $d$ uses resources determined in variable $y_{neks}$ at iterations $a_d + b_d \leq n \leq a_d + b_d + l_d$. So in iteration $n > a_d + b_d + l_d$ values of resources described by variable $y_{neks}$ are not used by this demand $d$.

### Implementation note

It is not necessary to store variable $y_{neks}$ for all $|N|$ iterations. In this method it is needed at $n'$-th iteration to know value of this variable for $n$-th iteration determined by: $n' \leq n \leq n' + l_{max}$, where $l_{max}$ is equal to number iterations of demand with the longest duration that is being processed in $n'$-th iteration. So, it is needed to store variable $y_{neks}$ for only $l_{max}$ iterations. This fact can avoid using too much program memory while implementing this method, because it is possible to reduce first dimension size from $|N|$ to $l_{max}$ by reindexing variable $y_{neks}$ after each iteration like in (\ref{aqn:reindexing}). After reindexing values of $y_{(l_{max} + 1)eks}$ are unknown, but it can be calculated using dependence described in (\ref{aqn:onechannel}). It is also needed to take into account that this number $l_{max}$ will vary in subsequent iterations.

$$
\begin{equation}
\label{aqn:reindexing}
\begin{split}
\forall_{n \in <1; l_{max}>} y_{neks} := y_{(n+1)eks}
\end{split}
\end{equation}
$$


## Research plan and simulation program

In order to test the possibilities of the method proposed by us in the previous chapter, it is necessary to create a tool that will enable simulations.

### Simulation program
Descibed method was implemented in created simulation program. We chose Python 3.9 with tool Numpy thanks to which we could easily store and manipulate data in arrays.

### Data sets
We also got a simulation data for traffics volumes from 100 E to 2000 E, 10 data sets for each traffic volume. The data covers 2000 iterations over a network with 16 nodes and 48 links. There are also 30 candidate paths between each two network nodes.

We can compare our results with the results of the SSA algorithm for 7 cores and 30 candidate paths between each two network nodes.

### Computational complexity

Each step of proposed method was implemented in main.py file. Unfortunately, with more requests coming in a given iteration or stored in assistive storage, the computation time is significantly longer. The size of the network (number of nodes, links and cores) and the number of candidate paths are also important.

The longest computation is for the value of $D_{eks}$ described in (\ref{aqn:deks}). This array is computed for each iteration and depends linearly on the number of links and cores, the number of demands and the number of candidate paths.

The second quantity that takes the longest time to compute is $c_d'$ described in (\ref{aqn:cd}). It is computed in each iteration as many times as the most allocable demand is searched for. The computation complexity depends linearly on the number of demands to check, the number of cores and the number of candidate paths.

### Research plan

Conducting a simulation for a network with the same parameters as the one for which we have the results of the SSA algorithm, takes too much time and exceeds our capabilities.

We decided to run simulations for networks with much lower parameters. In our research, we limited the number of cores to 2, and we limited the number of candidate paths to 3. As a result, the time needed to perform a single simulation ranges from 1 hour to 25 hours depending on the traffic volume (from about 2 seconds per iteration to about 40 seconds per iteration).

## Conducted research and results

The SSA algorithm for traffic volumes up to 1000 E did not report any rejected requests. The results may be different in a lower-performing network. We want to use our method to check whether any requests will be rejected without using assistive storage and how it will change with assistive storage of different sizes.  We will perform this test for traffic volumes of 300 E, 400 E, 500 E and 600 E. Thanks to this comparison, we will see how the use of assistive storage affects the allocation of requests.

![1800e results](./1800e.jpg) 

![1800e results](./2000e.jpg) 

| Traffic | Test | \| Pd \| | \| K \| | Storage | All bitrate | Served bitrate | Rejected bitrate | All demands | Served demands | Rejected demands | DBP    | BBP    |
|---------|------|----------|---------|---------|-------------|----------------|------------------|-------------|----------------|------------------|--------|--------|
| 300     | 1    | 3        | 2       | 0       | 15107220    | 14951401       | 155819           | 28976       | 28688          | 288              | 0,0099 | 0,0103 |
|         |      |          |         | 10      | 15107220    | 14951401       | 155819           | 28976       | 28688          | 288              | 0,0099 | 0,0103 |
|         |      |          |         | 30      | 15107220    | 14951401       | 155819           | 28976       | 28688          | 288              | 0,0099 | 0,0103 |
| 400     | 1    | 3        | 2       | 0       | 15104870    | 14900784       | 204086           | 28976       | 28594          | 382              | 0,0132 | 0,0135 |
|         |      |          |         | 10      | 15104870    | 14900784       | 204086           | 28976       | 28594          | 382              | 0,0132 | 0,0135 |
| 500     | 1    | 3        | 2       | 0       | 15097904    | 14839532       | 258372           | 28976       | 28491          | 485              | 0,0167 | 0,0171 |
|         |      |          |         | 10      | 15097904    | 14839532       | 258372           | 28976       | 28491          | 485              | 0,0167 | 0,0171 |
| 600     | 1    | 3        | 2       | 0       | 15098323    | 1493859        | 13604464         | 28976       | 28400          | 576              | 0,0199 | 0,9011 |
|         |      |          |         | 10      | 15098323    | 1493859        | 13604464         | 28976       | 28400          | 576              | 0,0199 | 0,9011 |
|         |      |          |         | 20      | 15098323    | 1493859        | 13604464         | 28976       | 28400          | 576              | 0,0199 | 0,9011 |
| 1000    | 1    | 3        | 2       | 0       | 15098139    | 12442521       | 2655618          | 28976       | 24991          | 3985             | 0,1375 | 0,1759 |
|         |      |          |         | 10      | 15098139    | 13660835       | 1437304          | 28976       | 26721          | 2255             | 0,0778 | 0,0952 |
|         |      |          |         | 20      | 15098139    | 14006028       | 1092111          | 28976       | 27210          | 1766             | 0,0609 | 0,0723 |
| 1300    | 1    | 3        | 2       | 0       | 15134248    | 11203183       | 3931065          | 28976       | 22937          | 6039             | 0,2084 | 0,2597 |
|         |      |          |         | 10      | 15134248    | 11258735       | 3875513          | 28976       | 23007          | 5969             | 0,2060 | 0,2561 |
|         |      |          |         | 20      | 15134248    | 11258735       | 3875513          | 28976       | 23007          | 5969             | 0,2060 | 0,2561 |
|         |      |          |         | 30      | 15134248    | 11258735       | 3875513          | 28976       | 23007          | 5969             | 0,2060 | 0,2561 |
| 1500    | 1    | 3        | 7       | 0       | 15170426    | 15136327       | 34099            | 28976       | 27475          | 1501             | 0,0518 | 0,0022 |
| 1800    | 1    | 3        | 2       | 0       | 30115206    | 8217437        | 21897769         | 28976       | 16278          | 12698            | 0,4382 | 0,7271 |
|         |      |          |         | 10      | 30115206    | 10881946       | 19233260         | 28976       | 21506          | 7470             | 0,2578 | 0,6387 |
|         |      |          |         | 20      | 30115206    | 10844820       | 19270386         | 28976       | 21550          | 7426             | 0,2563 | 0,6399 |
|         |      |          |         | 30      | 30115206    | 10851561       | 19263645         | 28976       | 21664          | 7312             | 0,2523 | 0,6397 |
|         |      |          |         | 40      | 30115206    | 10801027       | 19314179         | 28976       | 21645          | 7331             | 0,2530 | 0,6413 |
| 2000    | 1    | 3        | 2       | 0       | 15104625    | 9157159        | 5947466          | 28976       | 19466          | 9510             | 0,3282 | 0,3938 |
|         |      |          |         | 10      | 15104625    | 10313211       | 4791414          | 28976       | 21263          | 7713             | 0,2662 | 0,3172 |
|         |      |          |         | 20      | 15104625    | 10760596       | 4344029          | 28976       | 21919          | 7057             | 0,2435 | 0,2876 |

We also want to check what the results will be for higher traffic volumes. We chose 1000 E, 1300 E, 1800 E and 2000 E. Similar simulations will be performed to check how the number of rejected demands changes depending on the size of assistive storage.

We also conducted only one test with 7 cores for traffic volume of 1500 E, but with only 3 candidate paths for each demand. Thanks to it we can see, what is the different between our results and SSA algorithm's results, remembering about different number of candidate paths.

## Conclusions

The most important conclusion is that the designed method is not useful in real cases. The linear dependence of the calculation time on the network parameters allows only basic simulations to be made.

Other methods, such as linear programming, use matrix and number operations. Their computation times are much shorter. Our method requires searching resource and request sets, which greatly increases the number of instructions that the program has to execute.

Despite the few results obtained with our simulation program, we can draw conclusions regarding the use of assistive storage.


Based on the results obtained for low traffic:
* if the number of rejected demands without the use of assistive storage is low, using assistive storage does not improve the results. This is evidenced by the results obtained for the traffic volumes of 300 E - 600 E.


Based on the results obtained for high traffic:
* if the number of rejected demands without the use of assistive storage is high, the use of base assistive storage significantly reduces the number of rejected demands and reduces the DBP and BBP values. Further increasing the size of assistive storage only slightly improves the results. This is evidenced by the results obtained for the traffic volume of 1000 E, 1300 E and 1800 E,
* using assistive storage increases the number of operations and the computation time. This is because there are more candidate demands to allocate resources.

Based on the results obtained for 7 cores:
* the created method copes well with assigning resources to successive demands. The presented results are for networks with worse parameters. The result obtained for 1500 E rejected 1501 demands, and for the same data and the same network, but with 10 times more candidate paths, the SSA algorithm rejected 567 demands. We could not run the test for the 30 candidate paths as it would take 10 times longer.



## References
[1]: CISCO, “Cisco Annual Internet Report (2018-2023) White Paper,”CISCO, Tech. Rep., 2020.

[2]: W. KmiecikandK.Walkowiak,“A performance study of dynamic routing algorithm for sdm translucent optical networks with assistive storage,”Optical Switching and Networking,vol. 38, p. 100572, 2020. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S157342771930150X

[3]: M. Klinkowski,P. Lechowicz, and K. Walkowiak, “Survey of resource allocation schemes and algorithms in spectrally-spatially flexible optical networking,”Optical Switching and Networking, vol. 27, pp. 58 – 78, 2018. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S157342771730053X

[4]: B. Chatterjee, N. Sarma, and E. Oki, “Routing and spectrum allocationin elastic optical networks: A tutorial, ”IEEE Communications Surveysamp Tutorials, vol. 17, pp. 1776–1800, 07 2015.

[5]: International Telecommunication Union - ITU-T, “G.694.1 (02/2012), spectral grids for WDM applications: DWDM frequency grid,” Tech. Rep., 2012.

[6]: R. Rumipamba-Zambrano, F.-J. Moreno-Muro, J. Perelló, P. Pavón-Mariño, S. Spadaro, “Space continuity constraint in dynamic flex-grid/sdm optical core networks: An evaluation withspatial and spectral super-channels, ”Computer Communications,vol. 126, pp. 38–49, 2018. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S0140366417310162

[7]: M. Klinkowski and K. Walkowiak, “Routing and spectrum assignment in spectrum sliced elastic optical path network, ”IEEE Communications Letters, vol. 15, pp. 884–886, 08 2011.

[8]: A. Ghadesi, A. Ghaffarpour Rahbar, M. Yaghubi-Namaad, and A. Abi, “Intentional spectrum waste to reduce blocking probability in space division multiplexed elastic optical networks, ”Optical Fiber Technology, vol. 52, p. 101968, 2019. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S1068520019301051

[9]: R. Goścień and P. Lechowicz, “On the complexity of RSSA of anycast demands in spectrally-spatially flexible optical networks,” in INOC, 2019.

[10]: D. T. Hai, “On routing, spectrum and network coding assignment problem for transparent flex-grid optical networks with dedicated protection, ”Computer Communications, vol. 147, pp. 198 – 208, 2019. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S0140366418306546

[11]: M. Jinno, B. Kozicki, H. Takara, A. Watanabe, Y. Sone, T. Tanaka, and A. Hirano, “Distance-adaptive spectrum resource allocation in spectrum-sliced elastic optical path network [topics in optical communications], ”IEEE Communications Magazine, vol. 48, no. 8, pp. 138–145, 2010.

[12]: M. W. Przewozniczek, R. Goścień, P. Lechowicz, and K. Walkowiak, “Metaheuristic algorithms with solution encoding mixing for effective optimization of sdm optical networks, ”Engineering Applications of Artificial Intelligence, vol. 95, p. 103843, 2020. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S0952197620302074

[13]: P. Lechowicz, K. Walkowiak, and M. Klinkowski, “Greedy randomized adaptive search procedure for joint optimization of unicast and anycast traffic in spectrally-spatially flexible optical networks, ”Computer Networks, vol. 146, pp. 167 – 182, 2018. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S1389128618308995

[14]: R. Rumipamba-Zambrano, J. Perell ́o, J. M. Gen ́e, and S. Spadaro, “On the scalability of dynamic flex-grid/sdm optical core networks, ”Computer Networks, vol. 142, pp. 208 – 222, 2018. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S1389128618303712

[15]: Y. Qiu, “An efficient spectrum assignment algorithm based on variable-grouping mechanism for flex-grid optical networks, ”Optical Switching and Networking, vol. 24, pp. 39 – 46, 2017. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S1573427716300467

[16]: Y. Qiu and J. Xu, “Minimum-block-generated flexible-grouping-based spectrum assignment for flex-grid optical networks, ”Optical Fiber Technology, vol. 38, pp. 51 – 60, 2017. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S1068520017302572

[17]: A. Agrawal, U. Vyas, V. Bhatia, and S. Prakash, “Sla-aware differentiated qos in elastic optical networks, ”Optical FiberTechnology, vol. 36, pp. 41 – 50, 2017. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S1068520017300536

[18]: J. Mata, I. de Miguel, R. J. Dur ́an, N. Merayo, S. K. Singh, A. Jukan, and M. Chamania, “Artificial intelligence (ai) methods in optical networks: A comprehensive survey, ”Optical Switchingand Networking, vol. 28, pp. 43 – 57, 2018. [Online]. Available: http://www.sciencedirect.com/science/article/pii/S157342771730231X

In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>